<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/patch-1/Classification%20and%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import random
%matplotlib inline

In [0]:
from sklearn.preprocessing import LabelEncoder

# Getting the data

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
cd '/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean/'

/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean


In [0]:
all_data = pd.read_pickle('all_data.pkl')


# Model to search the visitors who made transactions

## Function to define Target Column

In [0]:
def is_transaction(n):
  if n == 0    :    return 0
  else         :    return 1

all_data['is_transaction']=all_data['totals.transactions'].apply(is_transaction)

In [9]:
all_data['is_transaction'].value_counts()

0    2085062
1      24864
Name: is_transaction, dtype: int64

## Encoding Categorical Values

In [0]:
TARGET = 'is_transaction'
CAT_COLS = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS = [c for c in all_data.columns if c not in CAT_COLS and c != TARGET]


In [13]:
CAT_COLS.remove('fullVisitorId')
CAT_COLS

['device.deviceCategory',
 'geoNetwork.country',
 'customDimension.value',
 'dataset',
 'geoNetwork.metro',
 'geoNetwork.continent',
 'trafficSource.medium',
 'device.operatingSystem',
 'geoNetwork.networkDomain',
 'channelGrouping',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.browser',
 'geoNetwork.subContinent',
 'geoNetwork.region',
 'trafficSource.source',
 'geoNetwork.city']

In [0]:
CONT_COLS.remove('date')

In [15]:
CONT_COLS.remove('totals.transactions')
CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS

['totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.bounces',
 'visitStartTime',
 'totals.pageviews',
 'totals.newVisits',
 'visitNumber',
 'trafficSource.isTrueDirect',
 'visitId',
 'device.isMobile',
 'totals.hits',
 'trafficSource.adwordsClickInfo.isVideoAd']

In [16]:
M1_data = deepcopy(all_data[CAT_COLS + CONT_COLS + [TARGET]+ [ 'fullVisitorId']])

M1_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 30 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   device.deviceCategory                         object 
 1   geoNetwork.country                            object 
 2   customDimension.value                         object 
 3   dataset                                       object 
 4   geoNetwork.metro                              object 
 5   geoNetwork.continent                          object 
 6   trafficSource.medium                          object 
 7   device.operatingSystem                        object 
 8   geoNetwork.networkDomain                      object 
 9   channelGrouping                               object 
 10  trafficSource.adwordsClickInfo.adNetworkType  object 
 11  device.browser                                object 
 12  geoNetwork.subContinent                       object 
 13

In [0]:
leb = LabelEncoder()
for col in CAT_COLS:
    M1_data[col] = leb.fit_transform(M1_data[col])

## Split Train and Test

In [16]:
M1_data['dataset'].value_counts()

1    1708337
0     401589
Name: dataset, dtype: int64

In [0]:
train = M1_data[M1_data['dataset']==1]
test = M1_data[M1_data['dataset']==0]
train.drop('dataset',axis=1,inplace=True)
test.drop('dataset',axis=1, inplace=True)

In [0]:
CAT_COLS.remove('dataset')

In [20]:
train.shape, test.shape

((1708337, 29), (401589, 29))

## Model

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import f1_score as f1
rmse = lambda true, pred: mse(true, pred) ** 0.5

In [0]:
x_train = train[CAT_COLS + CONT_COLS]
x_test = test[CAT_COLS + CONT_COLS]
y_train = train[[TARGET]]
y_test = test[[TARGET]]

In [26]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1708337, 27), (401589, 27), (1708337, 1), (401589, 1))

In [66]:
x_train.columns

Index(['device.deviceCategory', 'geoNetwork.country', 'customDimension.value',
       'geoNetwork.metro', 'geoNetwork.continent', 'trafficSource.medium',
       'device.operatingSystem', 'geoNetwork.networkDomain', 'channelGrouping',
       'trafficSource.adwordsClickInfo.adNetworkType', 'device.browser',
       'geoNetwork.subContinent', 'geoNetwork.region', 'trafficSource.source',
       'geoNetwork.city', 'totals.sessionQualityDim', 'totals.timeOnSite',
       'totals.bounces', 'visitStartTime', 'totals.pageviews',
       'totals.newVisits', 'visitNumber', 'trafficSource.isTrueDirect',
       'visitId', 'device.isMobile', 'totals.hits',
       'trafficSource.adwordsClickInfo.isVideoAd'],
      dtype='object')

In [0]:
model = RandomForestClassifier( 
    n_estimators=100, # Hyperparameter 1
    max_depth=10,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model = model.fit(x_train, y_train)

In [0]:
def get_error(pred, true):
    print(f'f-score:         {f1(true, pred)}')
    print(f'accuracy:        {accuracy(true, pred)}')

In [31]:
get_error(pred = trained_model.predict(x_test),
          true = y_test)

f-score:         0.09854067331310504
accuracy:   0.984464215902328


## Submission M1

In [32]:
pred_M1 = trained_model.predict(x_test)
pred_M1.shape

(401589,)

In [33]:
mylist = list(set(pred_M1))
mylist

[0, 1]

In [34]:
T_predicted = pred_M1[pred_M1!=0]
T_predicted.shape

(616,)

In [0]:
#train_id = train["fullVisitorId"].values
test_id = test["fullVisitorId"].values

In [106]:
sub_M1 = pd.DataFrame({"fullVisitorId":test_id})
sub_M1['PredictedLogRevenue']= pred_M1
sub_M1.shape

(401589, 2)

In [0]:
#sub_M2.append(sub_M1).drop_duplicates('fullVisitorId')

In [107]:
Cus_Pre = sub_M1[sub_M1['PredictedLogRevenue']==1]
test_id_M2 = list(Cus_Pre.index)
len(test_id_M2)

616

In [0]:
sub_M1.drop(test_id_M2,axis=0,inplace=True)
sub_M1.shape

# Model to calculate transaction Revenue

## Encoding Categorical Values - M2

In [0]:
TARGET_M2 = 'totals.totalTransactionRevenue'
CAT_COLS_M2 = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS_M2 = [c for c in all_data.columns if c not in CAT_COLS_M2 and c != TARGET_M2]


In [41]:
CAT_COLS_M2.remove('fullVisitorId')
CAT_COLS_M2

['device.deviceCategory',
 'geoNetwork.country',
 'customDimension.value',
 'dataset',
 'geoNetwork.metro',
 'geoNetwork.continent',
 'trafficSource.medium',
 'device.operatingSystem',
 'geoNetwork.networkDomain',
 'channelGrouping',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.browser',
 'geoNetwork.subContinent',
 'geoNetwork.region',
 'trafficSource.source',
 'geoNetwork.city']

In [42]:
#CONT_COLS.remove('totals.transactions')
#CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS_M2

['totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.bounces',
 'visitStartTime',
 'totals.transactions',
 'totals.pageviews',
 'date',
 'totals.newVisits',
 'visitNumber',
 'trafficSource.isTrueDirect',
 'visitId',
 'device.isMobile',
 'totals.hits',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'is_transaction']

In [43]:
M2_data = deepcopy(all_data[CAT_COLS_M2 + CONT_COLS_M2 + [TARGET_M2]+['fullVisitorId']])

M2_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 33 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   device.deviceCategory                         object        
 1   geoNetwork.country                            object        
 2   customDimension.value                         object        
 3   dataset                                       object        
 4   geoNetwork.metro                              object        
 5   geoNetwork.continent                          object        
 6   trafficSource.medium                          object        
 7   device.operatingSystem                        object        
 8   geoNetwork.networkDomain                      object        
 9   channelGrouping                               object        
 10  trafficSource.adwordsClickInfo.adNetworkType  object        
 11  device.browser           

In [0]:
leb = LabelEncoder()
for col in CAT_COLS_M2:
    M2_data[col] = leb.fit_transform(M2_data[col])

## Split Train and Test - M2

In [0]:
train_M2 = M2_data[M2_data['dataset']==1]
test_M2 = M2_data[M2_data['dataset']==0]
train_M2.drop('dataset',axis=1,inplace=True)
test_M2.drop('dataset',axis=1, inplace=True)

In [0]:
CAT_COLS_M2.remove('dataset')

In [48]:
train_M2.shape, test_M2.shape

((1708337, 32), (401589, 32))

In [0]:
#df[df['A'].isin([3, 6])]

In [68]:
P_test_M2 = test_M2[test_M2.index.isin(test_id_M2)]
P_test_M2.drop('is_transaction',axis=1,inplace=True)
P_test_M2.shape

(616, 31)

In [69]:
P_test_M2['fullVisitorId'].value_counts()

4099631678878053818    6
4608218212556854907    4
2239826195536738512    3
4428739076160836883    3
9249130983722228273    3
                      ..
7384784242167555295    1
6110416034079037629    1
6138331234857026828    1
9786595211770586469    1
8348476995025336130    1
Name: fullVisitorId, Length: 545, dtype: int64

But the training of the transaction revenue is made just with the data from visitors that made at least one transaction

In [0]:
T_train_M2 = train_M2[train_M2['is_transaction']==1]
#T_test_M2 = test_M2[test_M2['is_transaction']==1]
T_train_M2.drop('is_transaction',axis=1,inplace=True)
#T_test_M2.drop('is_transaction',axis=1, inplace=True)

In [72]:
CAT_COLS_M2

['device.deviceCategory',
 'geoNetwork.country',
 'customDimension.value',
 'geoNetwork.metro',
 'geoNetwork.continent',
 'trafficSource.medium',
 'device.operatingSystem',
 'geoNetwork.networkDomain',
 'channelGrouping',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.browser',
 'geoNetwork.subContinent',
 'geoNetwork.region',
 'trafficSource.source',
 'geoNetwork.city']

In [0]:
CONT_COLS_M2.remove('date')
CONT_COLS_M2.remove('is_transaction')

In [74]:
T_train_M2.shape, P_test_M2.shape

((18559, 31), (616, 31))

## Model - M2

In [0]:
x_train_M2 = T_train_M2[CAT_COLS_M2 + CONT_COLS_M2]
x_test_M2 = P_test_M2[CAT_COLS_M2 + CONT_COLS_M2]
y_train_M2 = T_train_M2[[TARGET_M2]]
y_test_M2 = P_test_M2[[TARGET_M2]]

In [76]:
x_train_M2.shape, x_test_M2.shape, y_train_M2.shape, y_test_M2.shape

((18559, 28), (616, 28), (18559, 1), (616, 1))

In [78]:
y_test_M2.describe()

,totals.totalTransactionRevenue
count,6.160000e+02
mean,1.264561e+08
std,4.001648e+08
min,0.000000e+00
25%,0.000000e+00
50%,2.609000e+07
75%,1.219900e+08
max,7.642380e+09


In [0]:
model_M2 = RandomForestRegressor( 
    n_estimators=100, # Hyperparameter 1
    max_depth=10,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model_M2 = model_M2.fit(x_train_M2, y_train_M2)

In [0]:
def get_error_regression(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')

In [82]:
get_error_regression(pred = trained_model_M2.predict(x_test_M2),
          true = y_test_M2)

RMSE:       471239550.3231417
MAE:        272812695.0264385
r2:         -0.3890292790826513


In [83]:
pred_M2 = trained_model_M2.predict(x_test_M2)
pred_M2.shape

(616,)

In [84]:
test_id_M2 = P_test_M2["fullVisitorId"].values
len(test_id_M2) #2896

616

In [85]:
sub_M2 = pd.DataFrame({'fullVisitorId':test_id_M2})
sub_M2['PredictedLogRevenue']= pred_M2
sub_M2

,fullVisitorId,PredictedLogRevenue
0,6533935296546803942,1.701716e+08
1,6915727589933825896,1.787653e+08
2,7026569611903775237,2.844919e+08
3,9086750370904150473,7.715089e+08
4,1517219381826577219,1.716856e+08
...,...,...
611,7911200327522123962,3.823872e+08
612,199211073374659585,2.836649e+08
613,6485327521360234229,3.670148e+08
614,9536260586427681418,5.096339e+08


In [110]:
sub_Tot=sub_M2.append(sub_M1)
sub_Tot

,fullVisitorId,PredictedLogRevenue
0,6533935296546803942,1.701716e+08
1,6915727589933825896,1.787653e+08
2,7026569611903775237,2.844919e+08
3,9086750370904150473,7.715089e+08
4,1517219381826577219,1.716856e+08
...,...,...
401584,6701149525099562370,0.000000e+00
401585,6154541330147351453,0.000000e+00
401586,6013469762773705448,0.000000e+00
401587,4565378823441900999,0.000000e+00


# build a submission file

In [0]:
sub_Tot["PredictedLogRevenue"] = np.expm1(sub_Tot["PredictedLogRevenue"])
sub_Tot = sub_Tot.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_Tot.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_Tot["PredictedLogRevenue"] = np.log1p(sub_Tot["PredictedLogRevenue"])
sub_Tot.to_csv("submission.csv", index=False)

In [113]:
sub_Tot.shape

(296530, 2)

In [128]:
sub_Tot[sub_Tot['PredictedLogRevenue']==0]

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.0
1,0000039738481224681,0.0
2,0000073585230191399,0.0
3,0000087588448856385,0.0
4,0000149787903119437,0.0
...,...,...
296525,9999862054614696520,0.0
296526,9999898168621645223,0.0
296527,999990167740728398,0.0
296528,9999915620249883537,0.0


In [0]:
# options for balancing classes
# - upsampling: resampling minority class until it is same quantity of majority class
# - downsampling: removing samples (randomly) from majority class unitl quantity same as minority
# - synthetic (smote): similar to upsampling, algorithmically create new minority samples

# choose a better metric for inbalanced classes:
# - f score

# ensembling methods:
# - classification model: make predictions about whether they made purchase (on test set)
# - regression model: predicted on dataset with no zeros

# options from there:
#  - just use these predictions
# - average out the results from regression model on all data and regression classification combo

In [0]:
from sklearn.metrics import f1_score